<a href="https://colab.research.google.com/github/dk-wei/ml-algo-implementation/blob/main/CountVectorizer%2BLogistic_Regression%2BELI5%E8%AE%B2%E8%A7%A3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

`CountVectorizer`用来对每个`document`进行`one-hot encoding`, 无论是对于NLP问题，还是多categorical feature情况，还是非常重要的。 

本文主要讲三个方面:
- `CountVectorizer`的各个parameter
- `tokenizer`用于split各个document (避免split合成词)，以及清除punctuation
- Logistic regression和eli5解释token importance

我们通过不同的参数进行比较

In [146]:
from sklearn.feature_extraction.text import CountVectorizer
import string
import pandas as pd

In [147]:
# Build our text
corpus = [
     'This is the first document.',
     'This document is the second-document.',
     'And this is the third-one.',
     'Is this the first_document?',
 ]

## 默认`CountVectorizer`

In [148]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

In [149]:
# 我们可以看到默认的tokenizer已经清洗了每个token周围的标点
print(vectorizer.get_feature_names())

['and', 'document', 'first', 'first_document', 'is', 'one', 'second', 'the', 'third', 'this']


In [150]:
print(X.toarray())

[[0 1 1 0 1 0 0 1 0 1]
 [0 2 0 0 1 0 1 1 0 1]
 [1 0 0 0 1 1 0 1 1 1]
 [0 0 0 1 1 0 0 1 0 1]]


## N-gram`CountVectorizer`

In [151]:
vectorizer2 = CountVectorizer(analyzer='word', 
                              ngram_range=(2, 2)
                              )

X2 = vectorizer2.fit_transform(corpus)

In [152]:
print(vectorizer2.get_feature_names())

['and this', 'document is', 'first document', 'is the', 'is this', 'second document', 'the first', 'the first_document', 'the second', 'the third', 'third one', 'this document', 'this is', 'this the']


In [153]:
print(X2.toarray())

[[0 0 1 1 0 0 1 0 0 0 0 0 1 0]
 [0 1 0 1 0 1 0 0 1 0 0 1 0 0]
 [1 0 0 1 0 0 0 0 0 1 1 0 1 0]
 [0 0 0 0 1 0 0 1 0 0 0 0 0 1]]


## `CountVectorizer` with new `tokenizer`

In [154]:
def tokenizer_splitter(s):
  '''
  按照space给split，再strip两边的punctuation
  '''

  res = [i.strip(string.punctuation) for i in s.split(' ')]
  res = [lemmatizer.lemmatize(i, 'v') for i in res if not i.isnumeric()]
  res = [lemmatizer.lemmatize(i, 'n') for i in res if not i.isnumeric()]

  return res
   
vectorizer3 = CountVectorizer(analyzer='word', 
                              ngram_range=(1, 1),
                              stop_words = ['is'],
                              binary = False,
                              lowercase = True,
                              #tokenizer = lambda x: x.split(" "),
                              tokenizer = tokenizer_splitter
                              )

In [155]:
X3 = vectorizer3.fit_transform(corpus)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['be'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [156]:
# 我们可以看到不会split合成词
print(vectorizer3.get_feature_names())

['and', 'be', 'document', 'first', 'first_document', 'second-document', 'the', 'third-one', 'this']


In [157]:
print(X3.toarray())

[[0 1 1 1 0 0 1 0 1]
 [0 1 1 0 0 1 1 0 1]
 [1 1 0 0 0 0 1 1 1]
 [0 1 0 0 1 0 1 0 1]]


In [158]:
print(vectorizer3.vocabulary_)   # vocabulary_则是告知了每个encoding每个位置上的token情况，要好好利用

{'this': 8, 'be': 1, 'the': 6, 'first': 3, 'document': 2, 'second-document': 5, 'and': 0, 'third-one': 7, 'first_document': 4}


In [159]:
corpus

['This is the first document.',
 'This document is the second-document.',
 'And this is the third-one.',
 'Is this the first_document?']

In [160]:
X3.todense()

matrix([[0, 1, 1, 1, 0, 0, 1, 0, 1],
        [0, 1, 1, 0, 0, 1, 1, 0, 1],
        [1, 1, 0, 0, 0, 0, 1, 1, 1],
        [0, 1, 0, 0, 1, 0, 1, 0, 1]])

## 查看CountVectorizer内部

`vectorizer3.vocabulary_`的`index`和`vectorizer3.get_feature_names()`的`value`是一致的

In [161]:
df_cvec = pd.DataFrame(X3.todense(),columns=vectorizer3.get_feature_names())
print (df_cvec.shape)
df_cvec.head()

(4, 9)


,and,be,document,first,first_document,second-document,the,third-one,this
0,0,1,1,1,0,0,1,0,1
1,0,1,1,0,0,1,1,0,1
2,1,1,0,0,0,0,1,1,1
3,0,1,0,0,1,0,1,0,1


In [162]:
vectorizer3.vocabulary_

{'and': 0,
 'be': 1,
 'document': 2,
 'first': 3,
 'first_document': 4,
 'second-document': 5,
 'the': 6,
 'third-one': 7,
 'this': 8}

In [163]:
vectorizer3.get_feature_names()

['and',
 'be',
 'document',
 'first',
 'first_document',
 'second-document',
 'the',
 'third-one',
 'this']

## 与Logistic Regression相结合

代码来源：https://colab.research.google.com/github/littlecolumns/ds4j-notebooks/blob/master/nyt-takata-airbags/notebooks/Airbag%20classifier%20search%20(CountVectorizer).ipynb#scrollTo=jrT5yB7CG7dv

把CountVectorizer和Logistic Regression一起的话，可以用token的weights来代表重要性

In [164]:
# Make data directory if it doesn't exist
!mkdir -p data
!wget -nc https://nyc3.digitaloceanspaces.com/ml-files-distro/v1/nyt-takata-airbags/data/sampled-labeled.csv -P data

File ‘data/sampled-labeled.csv’ already there; not retrieving.



In [165]:
import pandas as pd

# Allow us to display 100 columns at a time, and 100 characters in each column (instead of ...)
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_colwidth", 100)

In [166]:
labeled = pd.read_csv("data/sampled-labeled.csv")
labeled.head()

,is_suspicious,CDESCR
0,0.0,"ALTHOUGH I LOVED THE CAR OVERALL AT THE TIME I DECIDED TO OWN, , MY DREAM CAR CADILLAC CTS HAS T..."
1,0.0,"CONSUMER SHUT SLIDING DOOR WHEN ALL POWER LOCKS ON ALL DOORS LOCKED BY ITSELF, TRAPPING INFANT I..."
2,0.0,DRIVERS SEAT BACK COLLAPSED AND BENT WHEN REAR ENDED. PLEASE DESCRIBE DETAILS. TT
3,0.0,TL* THE CONTACT OWNS A 2009 NISSAN ALTIMA. THE CONTACT STATED THAT THE START BUTTON FOR THE IGNI...
4,0.0,THE FRONT MIDDLE SEAT DOESN'T LOCK IN PLACE. *AK


In [167]:
labeled = labeled.dropna()

In [168]:
labeled.is_suspicious.value_counts()

0.0    150
1.0     15
Name: is_suspicious, dtype: int64

In [169]:
train_df = pd.DataFrame({
    'is_suspicious': labeled.is_suspicious,
    'airbag': labeled.CDESCR.str.contains("AIRBAG", na=False).astype(int),
    'air bag': labeled.CDESCR.str.contains("AIR BAG", na=False).astype(int),
    'failed': labeled.CDESCR.str.contains("FAILED", na=False).astype(int),
    'did not deploy': labeled.CDESCR.str.contains("DID NOT DEPLOY", na=False).astype(int),
    'violent': labeled.CDESCR.str.contains("VIOLENT", na=False).astype(int),
    'explode': labeled.CDESCR.str.contains("EXPLODE", na=False).astype(int),
    'shrapnel': labeled.CDESCR.str.contains("SHRAPNEL", na=False).astype(int),
})
train_df.head()

,is_suspicious,airbag,air bag,failed,did not deploy,violent,explode,shrapnel
0,0.0,0,0,0,0,0,0,0
1,0.0,0,0,0,0,0,0,0
2,0.0,0,0,0,0,0,0,0
3,0.0,0,0,0,0,0,0,0
4,0.0,0,0,0,0,0,0,0


In [170]:
import nltk
nltk.download('wordnet')

import nltk
from nltk.stem import WordNetLemmatizer 

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [171]:
from sklearn.feature_extraction.text import CountVectorizer

def tokenizer_splitter(s):
  '''
  按照space给split，再strip两边的punctuation
  '''

  res = [i.strip(string.punctuation) for i in s.split(' ')]
  res = [lemmatizer.lemmatize(i, 'v') for i in res if not i.isnumeric()]
  res = [lemmatizer.lemmatize(i, 'n') for i in res if not i.isnumeric()]

  return res

vectorizer = CountVectorizer(
                              lowercase = True,
                              ngram_range = [1,1], 
                              binary=True,
                             tokenizer = tokenizer_splitter)

vectors = vectorizer.fit_transform(labeled.CDESCR)
vectors

<165x1916 sparse matrix of type '<class 'numpy.int64'>'
	with 8743 stored elements in Compressed Sparse Row format>

In [172]:
vectors.toarray()
#vectors.todense()

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [173]:
#vectorizer = CountVectorizer(binary=True)

vectors = vectorizer.fit_transform(labeled.CDESCR)
words_df = pd.DataFrame(vectors.toarray(), columns=vectorizer.get_feature_names())
words_df.head()

,,0-8,01v347000,02-04,02v105000,02v146000,03v455000,04/07/14,04/22/11,04/29/14,05/10/2013,05/2/2014,05v395000,07-24-2000,07/08/09.*jb,08v303000,09/24/08,"1,440.00",1-inch,"10,000",10/01/08,1000.00,12/13/09,12/15/06,12:00,12:oo,12th,"13,000","13,981","136,000",13v136000,14-above,15-25,160lbs,1996-2001,1:15pm,1st,2's,"2,775.98.*ak",2000.00,2001-2002,20k,20mph,25-30mph,2nd,3-4,3.6,3/28/13,3/4,30-35,...,why,wider,wife,wiggle,will,wilson,wind,window,windshield,wiper,wire,wish,with,within,without,withstand,witness,won't,wonder,woosh,word,work,work.*ak,worse,worsen,worst,worth,would,wouldn't,wrangler,wreck,wrist,write,wrong,x-ray,xterra,xxx,yard,yc,year,year-old,yes,yet,yield,york,you,your,zero,zone,¿fixed¿
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


**Big secret:** The "fit" part of .fit_transform means "learn the words." The "transform" part means "count them."

In [174]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

In [175]:
X = words_df
y = labeled.is_suspicious

#clf = RandomForestClassifier(n_estimators=100)
clf = LogisticRegression(C=1e9, solver='lbfgs')

clf.fit(X, y)

LogisticRegression(C=1000000000.0, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [176]:
y_true = y
y_pred = clf.predict(X)

matrix = confusion_matrix(y_true, y_pred)

label_names = pd.Series(['not suspicious', 'suspicious'])
pd.DataFrame(matrix,
     columns='Predicted ' + label_names,
     index='Is ' + label_names)

,Predicted not suspicious,Predicted suspicious
Is not suspicious,150,0
Is suspicious,0,15


In [177]:
clf.coef_[0]

array([-3.36758122, -0.0190115 , -0.02385632, ..., -0.11442133,
        1.23068858, -0.01100362])

In [178]:
token_dict = {i[1]: i[0] for i in vectorizer.vocabulary_.items()}

In [179]:
global_keywords = {}

for i in range(len(clf.coef_[0])):
    global_keywords[token_dict[i]] = clf.coef_[0][i]

In [180]:
# pos keyphrases
sorted(global_keywords.items(), key=lambda x:x[1], reverse = True)[:10]

[('burn', 3.9489243117163273),
 ('deploy', 3.8693031036243988),
 ("passenger's", 3.2703781570256405),
 ('both', 2.5856528926963396),
 ('face', 2.537107480033944),
 ('later', 2.2354278327444947),
 ('could', 2.1143810896077),
 ('fire', 2.040147222471495),
 ('hand', 2.024168297649442),
 ('passenger', 2.0172941257293506)]

In [181]:
#pip install eli5

In [182]:
import eli5
feature_names = list(X.columns)

# Use this line instead of warnings about judging these classifier
# eli5.show_weights(clf, feature_names=feature_names, show=eli5.formatters.fields.ALL)
eli5.show_weights(clf, feature_names=feature_names)

Weight?,Feature
+3.949,burn
+3.869,deploy
+3.270,passenger's
+2.586,both
+2.537,face
+2.235,later
+2.114,could
+2.040,fire
+2.024,hand
+2.017,passenger


eli5和logistic regression的结果是一致的，都是weights

### `RandomForestClassifier`

In [183]:
from sklearn.model_selection import train_test_split

X = words_df
y = labeled.is_suspicious

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y)

In [184]:
clf = RandomForestClassifier(n_estimators=100)

clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [185]:
y_true = y_test
y_pred = clf.predict(X_test)

matrix = confusion_matrix(y_true, y_pred)

label_names = pd.Series(['not suspicious', 'suspicious'])
pd.DataFrame(matrix,
     columns='Predicted ' + label_names,
     index='Is ' + label_names)

,Predicted not suspicious,Predicted suspicious
Is not suspicious,38,0
Is suspicious,4,0


In [186]:
eli5.show_weights(clf, feature_names=feature_names)

Weight,Feature
0.0347 ± 0.1786,burn
0.0240 ± 0.1635,both
0.0185 ± 0.1129,face
0.0183 ± 0.1068,1st
0.0152 ± 0.0855,sunroof
0.0123 ± 0.0771,pull
0.0120 ± 0.0826,hand
0.0119 ± 0.0739,degree
0.0118 ± 0.1105,fire
0.0111 ± 0.0654,passenger's


In [187]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(C=1e9, solver='lbfgs')

clf.fit(X_train, y_train)

LogisticRegression(C=1000000000.0, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [188]:
y_true = y_test
y_pred = clf.predict(X_test)

matrix = confusion_matrix(y_true, y_pred)

label_names = pd.Series(['not suspicious', 'suspicious'])
pd.DataFrame(matrix,
     columns='Predicted ' + label_names,
     index='Is ' + label_names)

,Predicted not suspicious,Predicted suspicious
Is not suspicious,38,0
Is suspicious,4,0


In [189]:
eli5.show_weights(clf, feature_names=feature_names, target_names=['not suspicious', 'suspicious'])

Weight?,Feature
+3.156,burn
+2.955,both
+2.736,face
+2.528,passenger's
+2.394,deploy
+2.386,apart
+2.386,rip
+2.386,sunroof
+2.227,break
+2.064,hand
